In [1]:
#import important packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
#set width of column
pd.options.display.max_colwidth = 200
pd.options.display.max_rows = 999
#suppress warning
import warnings
warnings.filterwarnings("ignore")
#translation api
from googletrans import Translator
import math

In [126]:
def deleteStepping(s,brand,stepping):
    length=len(stepping)-1
    
    start = s[s.find(brand)+length:].find(brand)+length
    
    return s[start:]

def splitC(s):
    ##make data consistent#
    s=" ".join(s.split())
    
    if 'Pentium(R)' in s and 'Intel(R) Pentium(R)' not in s:
        s= s.replace('Pentium(R)','Intel(R) Pentium(R)')
    elif 'Celeron(R)' in s and 'Intel(R) Celeron(R)' not in s:
        s= s.replace('Celeron(R)','Intel(R) Celeron(R)')
    elif 'Intel(R)' in s and 'Core(TM)' not in s and 'Core' in s and '-Core' not in s:
        s= s.replace('Core','Core(TM)')
    elif 'AMD64' in s and 'Quad-Core' in s:
        s=s.replace('Quad-Core','AMD Quad-Core')
    elif 'Quad-Core' in s and 'Processor' in s:
        s=s.replace('Quad-Core','AMD Quad-Core')
    else:
        s=s

        
    if 'AMD' in s and 'RADEON' in s:
        s= s.replace('RADEON','Radeon') 
    if 'AMD' in s and " APU with AMD " in s:
        s=s.replace(" with AMD "," ")
    if 'AMD' in s and " APU " in s:
        s=s.replace(" APU "," ")
    if 'AMD' in s and " with " in s:
        s=s.replace(" with "," ")
    if 'AMD' in s and " Graphics" in s:
        s=s.replace("Graphics","")
    if 'AMD' in s and "Radeon(TM)" in s:
        s=s.replace("Radeon(TM)","Radeon")
    if "Radeon(tm)" in s:
        s=s.replace("Radeon(tm)","Radeon")
    
    if 'i3 ' in s and 'i3-' not in s:
        s=s.replace('i3 ','i3-')
    elif 'i5 ' in s and 'i5-' not in s:
        s=s.replace('i5 ','i5-')
    elif 'i7 ' in s and 'i7-' not in s:
        s=s.replace('i7 ','i7-')
    else:
        s=s
        
    ###extract with comma
    if ',' in s:
        first=s.split(',')[0]
        second=s.split(',')[1].lstrip()
        if 'Intel64' in first and 'Intel64' in second:
            firstN = deleteStepping(first,'Intel','Intel64')
            secondN=deleteStepping(second,'Intel','Intel64')
            return (firstN,secondN)      
        elif 'Intel' in first and 'Intel' in second:
            start1 = first.find('Intel')
            start2 = second.find('Intel')
            return (first[start1:],second[start2:])
        elif 'AMD' in first  and ('CORES' in second.upper() or '+' in second):
            if 'AMD64' in first:
                return deleteStepping(first,'AMD','AMD64')
            else:
                start = first.find('AMD')
                s= first[start:]
                return s
        elif 'AMD64' in first and 'AMD64' in second:
            firstN = deleteStepping(first,'AMD','AMD64')
            secondN = deleteStepping(second,'AMD','AMD64')
            return (firstN,secondN)
        elif 'AMD' in second and 'AMD' in first:
            start1 = first.find('AMD')
            start2 = second.find('AMD')
            return (first[start1:],second[start2:])
        elif 'AMD' not in s and 'Intel' not in s:
            return 'Unusual'
        else:
            return 'Unusual'
    else:
        if 'Intel' in s and 'Intel64' in s:
            start = s[s.find('Intel64')+6:].find('Intel')+6
        elif 'Intel' in s and 'Genuine' in s:
            start = s.find('Genuine')
        elif 'Intel' in s:
            start = s.find('Intel')       
                    
        elif 'AMD64' in s:
            if 'AMD ' in s:
                start =s[s.find('AMD64')+4:].find('AMD')+4
            else:
                start=s.find('AMD64')
        elif ('x64' in s  or 'x86' in s) and 'AMD' in s:
            start =s.find('AMD')
        else:
            start=0
            
    s=" ".join(s.split())
       
    return s[start:]

def splitB(s):
    if len(s)!=2:
        if 'Intel' in s:
            if 'CPU' in s:
                s=s.replace('CPU',' ')
            if '@' in s:
                end = s.find('@')
            elif '-' in s:
                end = s.find('-')+5
            else:
                end=len(s)+1

            if 'Genuine' in s:
                start = s.find('Genuine')
            elif 'Intel(R)' in s:
                start = s.find('Intel(R)')+8
            else:
                start=0
            sn=s[start:end]
        else:
            sn=s
    else:
        return s
    
    ##correct Core(TM)2 E series
    if 'Core(TM)2' in s and 'Core(TM)2 Duo' not in s and 'Quad' not in s:
        sn=sn.replace('Core(TM)2','Core(TM)2 Duo')
    elif 'Core(TM) Duo' in s and 'Core(TM)2 Duo' not in s:
        sn=sn.replace('Core(TM) Duo','Core(TM)2 Duo')
    else:
        sn=sn
    sn=" ".join(sn.split()) 
    ###correct E series
    series = [' 4300',' 4400',' 6300',' 6320',' 6400',' 6420',' 6600',' 6700']


    if 'Core(TM)2 Duo'in sn:
        for sery in series:
            if 'Core(TM)2 Duo'+sery in sn:
                sn=sn.replace(sery,' E'+sery.lstrip())
                break
    return sn    

def getList(series,threshold):
    df = (100*series.value_counts()/len(series)).sort_values()
    listR = df[df.cumsum()>threshold].index
    return listR

def filterS(s,series,threshold):
    listR = getList(series,threshold)
    if s in listR:
        return s
    else:
        return 'Rare'
"""
def getC(df,threshold):
    df=(100*df.brand.value_counts()/len(df.brand)).sort_values()
    listR=df[df.cumsum()>threshold].index
    return listR

###filter the RAM you want. Otherwise, it will be treated as "Rare". Criteria is the cumulative proportion.
def filterC(s,listC):
    if s in listC:
        return s
    else:
        return 'Rare'
"""  

def consis(s):
    if 'Intel' in s and 'Intel(R)' not in s:
        s=s.replace('Intel','Intel(R)')
    if 'Intel(R)' in s and 'Intel(R) -' in s:
        s=s.replace('Intel(R) -','Intel(R)')
    if 'Intel(R)' in s and 'Family ' in s:
        s=s.replace('Family ','Family')
    if 'Intel' in s and '(PE) ' in s:
        s=s.replace('(PE) ','')
    if 'Intel' in s and '(PE64) ' in s:
        s=s.replace('(PE64) ','')
    
    if 'Radeon(TM)' in s:
        s=s.replace('Radeon(TM)','Radeon')
    if 'Radeon (TM)' in s:
        s=s.replace('Radeon (TM)','Radeon')
    if 'RADEON' in s:
        s=s.replace('RADEON','Radeon')
    if 'Radeon.' in s:
        s=s.replace('Radeon.','Radeon')
    if 'RadeonT' in s:
        s=s.replace('RadeonT','Radeon')
    
    if 'Radeon' in s and 'AMD' not in s and 'ATI' not in s:
        s=s.replace('Radeon','AMD(ATI) Radeon')
    if 'ASUS' in s and 'Radeon' in s:
        s=s.replace('ASUS','')
    if 'Intel' in s and 'Iris' in s and 'Iris(TM)' not in s:
        s=s.replace('Iris','Iris(TM)')
    if 'Intel' in s and 'Iris(R)' in s:
        s=s.replce('Iris(R)','Iris(TM)')
    if 'ATI' in s and 'AMD' not in s:
        s=s.replace('ATI','AMD(ATI)')
    if 'AMD' in s and 'ATI' not in s:
        s=s.replace('AMD','AMD(ATI)')
    
    # delete redundant info
    if 'AMD' in s or 'Intel(R)' in s or 'NVIDIA' in s:
        if '(Microsoft Corporation - WDDM 1.0)' in s:
            s=s.replace('(Microsoft Corporation - WDDM 1.0)','')
        elif '(Microsoft Corporation - WDDM 1.1)' in s:
            s=s.replace('(Microsoft Corporation - WDDM 1.1)','') 
        elif '(Microsoft Corporation - WDDM 1.2)' in s:
            s=s.replace('(Microsoft Corporation - WDDM 1.2)','')
        elif '(Microsoft Corporation — WDDM 1.2)' in s:
            s=s.replace('(Microsoft Corporation — WDDM 1.2)','')
        elif '(Microsoft Corporation — WDDM 1.2)' in s:
            s=s.replace('(Microsoft Corporation — WDDM 1.2)','')
        elif '(Microsoft Corporation - WDDM v1.1)' in s:
            s=s.replace(' (Microsoft Corporation - WDDM v1.1)','')
        elif '(Microsoft Corporation WDDM 1.1)' in s:
            s=s.replace('(Microsoft Corporation WDDM 1.1)','')
        elif '(Microsoft Corporation - WDDM v1.20)' in s:
            s=s.replace('(Microsoft Corporation - WDDM v1.20)','')
        elif '(Microsoft Corporation - WDDM v1.2)' in s:
            s=s.replace('(Microsoft Corporation - WDDM v1.2)','')
        elif '(Engineering Sample - WDDM v1.1)' in s:
            s=s.replace('(Engineering Sample - WDDM v1.1)','')
        elif '(Microsoft Corporation - WDDM)' in s:
            s=s.replace('(Microsoft Corporation - WDDM)','')
        elif '(Microsoft Corporation- WDDM v1.20)' in s:
            s=s.replace('(Microsoft Corporation- WDDM v1.20)','')
        elif '(Bootcamp XG edition by bootcampdrivers.com)' in s:
            s=s.replace('(Bootcamp XG edition by bootcampdrivers.com)','')
        elif '(Engineering Sample - WDDM v1.20)' in s:
            s=s.replace('(Engineering Sample - WDDM v1.20)','')
        else:
            s=s
    # Clean ASUS
    if 'ASUS' in s:
        if 'EAH' in s:
            s=s.replace('ASUS ','AMD(ATI) Radeon ').replace('EAH', 'HD ').replace(' Series','') 
        elif 'HD' in s:
            s=s.replace('ASUS ','AMD(ATI) Radeon ').replace('HD','HD ').replace(' Series','')
        elif 'R5' in s or 'R7' in s or 'R9' in s:
            s=s.replace('ASUS','AMD(ATI) Radeon ').replace(' Series','')
        else:
            s='UNKNOWN'
        s=" ".join(s.split())
    
    if 'AMD' in s and (' Series' in s or ' Graphics' in s or ' series' in s or ' graphics' in s or ' Graphic' in s):
        s=s.replace(' Series','')
        s=s.replace(' series','')
        s=s.replace(' Graphics','')
        s=s.replace(' graphics','')
        s=s.replace(' Graphic','')

        
    s=" ".join(s.split())
    return s                                                

        

def cleanGC(s):
    if isinstance(s, float)==True:
        return s
    
    s=consis(s)
    
    if ',' in s:
        names = s.split(',')
        new = []
        for name in names:
            if '\\' not in name and 'PCI' not in name and 'ROOT' not in name and '%' not in name:
                new.append(name)
        s= ",".join(new)
    elif s[0]=='@' or s[0]=='\\' or s.startswith('PCI'):
        s='UNKNOWN'
        
    
    if ';' in s:
        names = s.split(';')
        new = []
        for name in names:
            if '%' not in name and '@' not in name:
                new.append(name)
        s=";".join(new)
    if s=='' or s[0]=='@' or s[0]=='%':
        return 'UNKNOWN'
    return s

def lzytrans(s):
    if isinstance(s, float)==True:
        return s  
    if ('Microsoft' in s or 'Майкрософт' in s) and ('Adaptador' in s or '适配器' in s or 'видеоадаптер' in s or '기본 디스플레이 어댑터' in s or 'Vídeo Básico' in s or 
                               'Görüntü Bağdaştırıcısı' in s or '基本顯示卡' in s or 'vidéo' in s or 'ディスプレイ アダプター' in s or 
                              'Základní grafický adaptér' in s or 'Placa Gráfica Básica da' in s or 'Microsoftin perusnäyttösovitin' in s or
                            'Microsoft Basic-skærmkort' in s or 'Βασικός προσαρμογέας οθόνης Microsoft' in s or 'Базовый видеоадаптер' in s or
          'videokártya' in s or '기본 디스플레이 드라이버' in s or 'Podstawowa karta graficzna' in s or 'Scheda video' in s):
        s= 'Microsoft Basic Display Adapter'
    elif 'Hyper-V' in s or 'Video för Hyper-V från Microsoft' in s:
        s= 'Microsoft Hyper-V-Video'
    elif 'VGA' in s and 'VMware' not in s and 'Wifi' not in s:
        s='Standard VGA Graphics Adapter'
    elif 'Семейство наборов микросхем Intel(R) G33/G31 Express' in s or 'Intel(R) G33/G31 Express Yonga Kümesi Ailesi' in s\
          or 'Familia Intel(R) G33/G31 Express Chipset' in s:
        s='Intel(R) G33/G31 Express Chipset Family'
    elif 'Набор микросхем Intel(R) G41 Express' in s or 'Intel(R) Express Chipset G41' in s:
        s='Intel(R) G41 Express Chipset'
    elif 'Mikroukład Intel(R) Q45/Q43 Express Chipset' in s or 'Набор микросхем Intel(R) Q45/Q43 Express' in s\
          or 'Intel(R) Express Chipset Q45/Q43' in s:
        s='Intel(R) Q45/Q43 Express Chipset'
    elif 'Семейство наборов микросхем Mobile Intel(R) 45 Express' in s or 'Mobile Intel(R) 45 Express lapkakészletcsalád' in s\
          or 'Familia Mobile Intel(R) 45 Express Chipset' in s:
        s='Mobile Intel(R) 45 Express Chipset Family'

    elif 'Familia Intel(R) 82945G Express Chipset' in s:
        s='Intel(R) 82945G Express Chipset Family'
    elif 'Familia Mobile Intel(R) 965 Express Chipset' in s:
        s='Mobile Intel(R) 965 Express Chipset Family '
    elif 'Intel(R) - famiglia Express Chipset Q965/Q963' in s:
        s='Intel(R) Q965/Q963 Express Chipset'
    elif 'Familia' in s:
        s=s.replace('Familia','')+' Family'
    else:
        s=s
    
    """
    # takes long time to run
    if isEnglish(s)==False:
        translator = Translator()
        t=translator.translate(s).text.title()
        s=t
    """
    if 'корпорация Майкрософт' in s:
        s=s.replace('корпорация Майкрософт','Microsoft Corporation')
    return s

def solveVM(s):
    if isinstance(s, float)==True:
        return s
    elif 'VGA' in s.upper():
        if 'VMware' in s:
            return 'VMware SVGA 3D'
        elif 'Standard' in s or 'Scheda' in s or 'Standardowa' in s or 'standard' in s:
            return 'Standard VGA Graphics Adapter'
        elif 'Wifi' in s:
            return 'Wireless'
        else:
            return s
    elif 'Microsoft' in s and ('Basic' in s or 'Base' in s):
        return 'Microsoft Basic Display Adapter'
    elif 'Microsoft' in s and 'Hyper' in s:
        return 'Microsoft Hyper-V Video'
    elif 'Intel' in s and 'Intel(R)' not in s:
        s=s.replace('Intel','Intel(R)')
        if 'Iris' in s and 'Iris(TM)' not in s:
            s=s.replace('Iris','Iris(TM)')
        elif 'Iris(R)' in s:
            s=s.replce('Iris(R)','Iris(TM)')
        if s.startswith('The'):
            s= s.replace('The','')
        return s
    elif 'Mirage' in s or 'Mirror' in s or 'BB Capture Driver' in s:
        return 'Mirror'
    elif 'Remote' in s or 'RC' in s:
        return 'Remote'
    elif 'DisplayLink USB' in s or 'Projector' in s:
        return 'Wireless'
    elif 'Parallels Display' in s or 'hook driver' in s or 'Citrix Display' in s or 'ShoreTel Desktop' in s\
          or 'VirtualBox Graphics' in s or 'Fujitsu Live Help graphics driver' in s or \
          'Freedom Scientific Accessibility' in s or 'ASPEED Graphics Family' in s:
        return 'Remote'
    else:
        s=s
    s=" ".join(s.split())
    return s

##### clean the duplicate GPU info
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

def mergeGPU(row):
    s=row['GC']
    
    if isinstance(s, float)==True:
        return s
    
    if ',' in s and row['CPU#']==1:
        if s.count('Intel(R)')>=2:
            start=find_nth(s,'Intel',2)
            s=s[:start]
        elif s.count('NVIDIA')>=2:
            start=find_nth(s,'NVIDIA',2)
            s=s[:start]
        elif s.count('AMD')>=2:
            start=find_nth(s,'AMD',2)
            s=s[:start]
        else:
            return 'Duplicate GPU'
    elif ',' in s and row['CPU#']==2:
        return '2 GPUs'
    else:
        s=s
    s=" ".join(s.split())
    return s

def loadGC(s):
    #consistent and clean
    s=cleanGC(s)
    s=lzytrans(s)
    s=solveVM(s)
    return s

def typeGPU(s):
    if isinstance(s, float)==True:
        return 'UNKNOWN'
    
    if 'Intel' in s:
        return 'Intel'
    elif 'AMD' in s:
        return 'AMD'
    elif 'NVIDIA' in s:
        return 'NVIDIA'
    elif 'Microsoft Basic Display' in s:
        return s
    elif 'Microsoft Hyper-V Video' in s:
        return s
    elif 'Mirror' in s:
        return 'Mirror'
    elif 'Remote' in s:
        return 'Remote'
    elif 'VMware' in s:
        return s
    elif 'UNKNOWN' in s:
        return s
    elif 'Wireless' in s:
        return s
    elif 'Matrox' in s:
        return 'Matrox'
    elif 'Standard VGA' in s:
        return 'Standard VGA Graphics Adapter'
    elif 'Windows Live Display Driver' in s:
        return s
    else:
        return 'Others'

def splitR(s):
    if s =='UNKNOWN':
        return np.nan
    elif 'GB' in s:
        return s[:s.find('GB')]
    else:
        return 'Others'

""" 
def getL(df,threshold):
    df=(100*df.RAM.value_counts().sort_index()/len(df.RAM)).sort_values()
    listR=df[df.cumsum()>threshold].index
    return listR

###filter the RAM you want. Otherwise, it will be treated as "Rare". Criteria is the cumulative proportion.
def filterR(s,listR):
    if s in listR:
        return s
    else:
        return 'Rare'
 """ 

def findOS(s):

    if 'WINDOWS' in s.upper() or 'enterprise' in s or 'Microsoft' in s or 'Майкрософт' in s:
        
        x='Windows'
    
        if 'pro' in s.lower() or 'professional' in s.lower() or '专业' in s or '專業' in s or 'Профессиональная' in s or 'Altum' in s:
            return x+' Pro'
        elif 'Home' in s or '家庭' in s or '家用' in s or 'Famille' in s or 'Домашняя' in s or 'Familiale' in s:
            return x+' Home'
        elif '企业' in s or 'entreprise' in s.lower() or '企業' in s or 'Корпоративная' in s or 'enterprise' in s.lower():
            return x+' Entreprise'
        elif 'education' in s.lower() or '教育' in s or 'Éducation' in s: 
            return x+' Education'
        elif '旗舰' in s or 'ultimate' in s.lower() or 'Максимальная' in s or '旗艦' in s or 'Intégrale' in s:
            return x+' Ultimate'
        elif 'CRUX' in s:
            return x+' Crux'
        elif 'Windows Server 2012 R2' in s:
            return x+' Server 2012 R2'
        elif 'Windows Server 2016 ' in s:
            return x+' Server 2016'
        elif 'Windows Server 2012' in s and 'R2' not in s:
            return x+' Server 2012'
        elif 'Windows Server 2008 R2' in s:
            return x+' Server 2008 R2'
        elif 'Starter' in s or 'Начальная' in s or '简易' in s:
            return x+' Starter'
        elif 'Embedded' in s:
            return 'NA'
        elif 'Luxury' in s:
            return x+' Luxury'
        elif ' MultiPoint Server 2012' in s:
            return x+' MultiPoint Server 2012'
        else:
            return x+' Standard'  # without knowledge of which edition
    elif 'MAC' in s.upper() or 'Yosemite' in s or 'EI Capitan' in s:
        return 'Mac'
    elif 'UNKNOWN' in s:
        return 'UNKNOWN'
    else:
        return 'Others'

def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start


def splitone(s):
    if 'Retina' in s:
        return s[:s.find('Retina')+6]
    elif 'Touch' in s:
        return s[:s.find('Touch')+5]
    else:
        return s.split()[0]
    
def splittwo(s):    
    splits = s.split(',')
    first = splits[0]
    last = splits[-2]
    
    s1=splitone(first)
    s2=splitone(last)
    
    return (s1,s2)
"""  
def getS(df,threshold):
    df=(100*df.Screen.value_counts()/len(df.Screen)).sort_values()
    listR=df[df.cumsum()>threshold].index
    return listR

###filter the RAM you want. Otherwise, it will be treated as "Rare". Criteria is the cumulative proportion.
def filterS(s,listR):
    if s in listR:
        return s
    else:
        return 'Rare'    
""" 
 

def cleandata(df,filterCPU=100, filterGPU=100,filterRAM=100,filterScreen=100):
    user=df
    user['Name']=user.device_hw_proc.apply(splitC)
    ins=''
#user['CPU#']=user['Name'].apply(lambda x: 2 if len(x)==2 else 1)
    
    user['CPU#']=user['Name'].apply(lambda x: 2 if len(x)==2 else 1)

    user['brand']=user.Name.apply(splitB)
    user.drop('Name',axis=1,inplace=True)
    """
    if filterCPU<100:
        ins=ins+'CPU has been filtered'
        listC = getC(user,filterCPU)
        user['brand']=user['brand'].apply(lambda x:filterR(x,listC))
    """
    user['GC']=user.device_hw_video.apply(loadGC)
    user['GC']=user.apply(mergeGPU,axis=1)
    
    ##if sum(user.GC.apply(typeGPU)=='Others') >filterGPU:
     ##   ins=ins+'GPU has been filtered'
      ###  user.loc[user.GC.apply(typeGPU)=='Others','GC']='Rare'
    
    
    user['RAM'] = user['device_hw_ram'].apply(splitR)
    
    
    """
    if filterRAM<100:
        ins+'RAM has been filtered'
        listR = getL(user,filterRAM)
        user['RAM']=user['RAM'].apply(lambda x:filterR(x,listR))
    """
    user['device_os_name'] = user['device_os_name'].apply(lambda x: " ".join(x.split()))
    user['OS'] = user.device_os_name.apply(findOS)
    user['Version']=user.device_os_version.str.split().str[0]

    user['Version2']=user.Version.apply(lambda x: x[:find_nth(x,'.',2)])

    ##integrate os and version2
    user['OS2']=user['OS']+' '+user['Version2']   
    
    user['ScreenQ']=user.device_hw_display.str.count('x')
    user.loc[user.device_hw_display.isnull(),'ScreenQ']=0
    user['Screen']=np.nan
    #user['Touch']=np.nan
    user.loc[user.ScreenQ==1, 'Screen'] = user.loc[user.ScreenQ==1, 'device_hw_display'].apply(splitone)
    user.loc[user.ScreenQ==2, 'Screen'] = user.loc[user.ScreenQ==2, 'device_hw_display'].apply(splittwo)
    user.loc[user.ScreenQ==0, 'Screen'] = 'No Screen'
    #user.loc[user.ScreenQ==0, 'Touch'] = 'No Screen'
    """
    if filterScreen<100:
        ins+'Screen has been filtered'
        listS = getS(user,filterScreen)
        user['Screen']=user['Screen'].apply(lambda x:filterS(x,listS))
#ROV.loc[ROV.device_hw_display.isnull(),'Touch']=ROV.loc[ROV.device_hw_display.isnull(),'device_hw_display'].apply(lambda x: 'Yes' if 'Touch' in x else 'No')
    """
    
    if filterCPU<100:
        user['brand']=user['brand'].apply(lambda x: filterS(x,user['brand'],filterCPU))
        ins=ins+'CPU has been filtered'
    if filterGPU<100:
        user['GC']=user['GC'].apply(lambda x: filterS(x,user['GC'],filterGPU))
        ins=ins+'GPU has been filtered'
    if filterRAM<100:
        ins=ins+'RAM has been filtered'
        user['RAM']=user['RAM'].apply(lambda x: filterS(x,user['RAM'],filterRAM))
    if filterScreen<100:
        ins=ins+'Screen has been filtered'
        user['Screen']=user['Screen'].apply(lambda x: filterS(x,user['Screen'],filterScreen))
    #user.loc[user.device_hw_display.notnull(),'Touch']=user.loc[user.device_hw_display.notnull(),'device_hw_display'].str.contains('Touch')
    
    user_test=user[['device_hw_proc','brand','device_hw_video','GC','device_hw_ram','RAM','device_os_name','device_os_version','OS2','device_hw_display','Screen']]
    return ins,user_test

In [187]:
#Read CSV
#test using 20_24
file = '2017_12_13_00_09.csv'
fake= pd.read_csv(file)
fake=fake.drop(['device_type','product_name','event_id','device_id','device_hw_form'],axis=1)
## drop row whose info is almost lost
fake=fake[fake['device_hw_ram'].notnull()].sample(10000)
fake.head()

,device_hw_proc,device_hw_ram,device_hw_display,device_hw_video,device_os_name,device_os_version
292662,Intel64 Family 6 Model 94 Stepping 3 Intel(R) Core(TM) i5-6500 CPU @ 3.20GHz,16GB 1073741824,"1920x1080 96,96 32, 1920x1080 96,96 32","Intel(R) HD Graphics 530, PCI\VEN_8086&DEV_1912&SUBSYS_8056103C&REV_06\3&11583659&1&10, \\.\DISPLAY1",Microsoft Windows 10 Pro,10.0.14393 Build 14393
336254,Intel64 Family 6 Model 58 Stepping 9 Intel(R) Core(TM) i5-3210M CPU @ 2.50GHz,8GB 0,"1024x768 96,96 32","Microsoft Basic Display Adapter, PCI\VEN_1002&DEV_6841&SUBSYS_90AB104D&REV_00\4&8175F2C&0&0008",Microsoft Windows 10 Pro,10.0.15063 Build 15063
583621,Intel64 Family 6 Model 30 Stepping 5 Intel(R) Core(TM) i5 CPU 750 @ 2.67GHz,8GB 1073741824,"1920x1080 96,96 32, 1920x1080 96,96 32","NVIDIA Quadro 600, PCI\VEN_10DE&DEV_0DF8&SUBSYS_083510DE&REV_A1\4&113C73B2&0&0018, \\.\DISPLAY1",Microsoft Windows 7 Максимальная,6.1.7601 Service Pack 1 Build 7601
18118,AMD FX(tm)-8350 Eight-Core Processor,16GB,1920x1080,AMD Radeon R7 200 Series,Microsoft Windows 8.1 Pro,6.3.9600 Build 9600
197106,x64 Family 6 Model 37 Stepping 5 Intel(R) Core(TM) i7 CPU M 620 @ 2.67GHz,3GB 14303204242096128,"1600x900 96,96 32","NVIDIA NVS 5100M, PCI\VEN_10DE&DEV_0A2C&SUBSYS_1521103C&REV_A2\4&2CA8BF17&0&0008",Microsoft Windows 7 Professional,6.1.7601 Service Pack 1 Build 7601


In [188]:
ins,fake = cleandata(fake,filterCPU=5, filterGPU=2,filterRAM=2,filterScreen=5)
fake=fake.drop(['device_hw_proc','device_hw_video','device_hw_ram','device_os_name','device_os_version','device_hw_display'],axis=1)
fake=fake.dropna()
#fake=fake[(fake.RAM!='Rare')&(fake.brand!='Rare')&(fake.Screen!='Rare')&(fake.GC!='UNKNOWN')]

In [143]:
fake.RAM.value_counts()

8       3845
16      2102
4       1751
32       522
3        445
12       304
6        271
Rare     166
2        150
7        133
24        60
64        53
Name: RAM, dtype: int64

In [144]:
#fake.GC.value_counts()
ins
#fake.head()

'CPU has been filteredRAM has been filteredScreen has been filtered'

In [199]:
fake=fake[(fake.RAM!='Rare')&(fake.brand!='Rare')&(fake.Screen!='Rare')&(fake.GC!='UNKNOWN')&(fake.GC!='Rare')]

In [190]:
fake.info()
#fake = fake.sample(10000,random_state=25)
#fake.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8876 entries, 292662 to 344012
Data columns (total 5 columns):
brand     8876 non-null object
GC        8876 non-null object
RAM       8876 non-null object
OS2       8876 non-null object
Screen    8876 non-null object
dtypes: object(5)
memory usage: 416.1+ KB


In [347]:
#user=pd.get_dummies(fake,['brand','GC','RAM','OS2','Screen'])
user1=pd.get_dummies(fake,columns=['brand']).mul(3)
user2=pd.get_dummies(fake,columns=['GC']).mul(1)
user3=pd.get_dummies(fake,columns = ['RAM','OS2','Screen']) 
user=pd.concat([user1,user2,user3],axis=1).drop(['RAM','OS2','Screen','GC','brand','labelP','labelP1','labelP2','labelP10','CGlabel','labelP30'],axis=1)

In [348]:
user.head()

,"brand_('Intel(R) Xeon(R) CPU E5-2667 0 @ 2.90GHz', 'Intel(R) Xeon(R) CPU E5-2667 0 @ 2.90GHz')","brand_('Intel(R) Xeon(R) CPU E5620 @ 2.40GHz', 'Intel(R) Xeon(R) CPU E5620 @ 2.40GHz')",brand_AMD A10-5745M Radeon HD,brand_AMD A10-5750M Radeon HD,brand_AMD A10-6800K Radeon HD,brand_AMD A10-7300 Radeon R6,brand_AMD A10-7800 Radeon R7,brand_AMD A10-7850K Radeon R7,brand_AMD A10-7860K Radeon R7,brand_AMD A10-8700P Radeon R6,...,Screen_1440x900,Screen_1600x1200,Screen_1600x900,Screen_1680x1050,Screen_1920x1080,Screen_1920x1080 Touch,Screen_1920x1200,Screen_2560x1080,Screen_2560x1440,Screen_3840x2160
292662,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
336254,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
583621,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18118,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
476980,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [349]:
user.shape
#pd.DataFrame([[1,0],[0,1]]).mul(3)
#pd.get_dummies(fake,columns=['brand','GC'])

(8556, 777)

In [350]:
from sklearn.cluster import KMeans
KMeans = KMeans(n_clusters=15)
KMeans.fit(user)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=15, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [351]:
fake['labelK15']=KMeans.labels_

In [371]:
fake.labelK15.value_counts()

1     2454
3     1480
0     1106
13     862
5      398
7      280
12     266
6      263
4      260
2      254
14     242
9      237
10     215
11     174
8       65
Name: labelK15, dtype: int64

In [381]:
pd.DataFrame(fake[fake['labelK15']==0].groupby(['brand','GC','RAM','OS2','Screen']).labelK15.count()).sort_values('labelK15')

labelK15
brand                                   GC                                        RAM OS2                    Screen                          
AMD A6-6400K Radeon HD                  AMD(ATI) Radeon HD 8470D                  3   Windows Pro 6.1        1920x1080                      1
Core(TM) i7-4770                        NVIDIA GeForce GTX 650 Ti                 32  Windows Pro 6.1        1920x1080                      1
                                                                                  8   Windows Pro 6.1        1920x1080                      1
                                        NVIDIA GeForce GTX 660 Ti                 8   Windows Pro 6.1        1920x1080                      1
                                        NVIDIA GeForce GTX 750                    16  Windows Pro 6.1        1920x1080                      1
                                        NVIDIA GeForce GTX 760 (192-bit)          32  Windows Pro 6.1        (1920x1080, 1920x1080)         1
                                        NVIDIA NVS 510                            32  Windows Pro 6.1        (1920x1080, 1920x1080)         1
                                        NVIDIA Quadro K2000D                      16  Windows Pro 6.1        (1920x1200, 1920x1200)         1
                                        NVIDIA Quadro K4200                       24  Windows Pro 6.1        1920x1080                      1
                                        NVIDIA Quadro K600                        12  Windows Pro 6.1        1920x1080                      1
                                                                                  16  Windows Pro 6.1        1600x900                       1
                                                                                                             1920x1080                      1
                                        NVIDIA GeForce GTX 650 Ti                 16  Windows Pro 6.1        1920x1080                      1
                                        NVIDIA Quadro K600                        32  Windows Pro 6.1        1920x1080                      1
                                                                                  8   Windows Pro 6.1        1920x1200                      1
                                        NVIDIA Quadro K620                        16  Windows Pro 6.1        (1920x1080, 1920x1080)         1
                                        Remote                                    8   Windows Pro 6.1        1920x1200                      1
Core(TM) i7-4770K                       AMD(ATI) Radeon HD 5450                   16  Windows Pro 6.1        (1920x1080, 1920x1080)         1
                                        NVIDIA Quadro K2000                       16  Windows Pro 6.1        1920x1080                      1
Core(TM) i7-4771                        NVIDIA GeForce GTX 660                    16  Windows Pro 6.1        1920x1080                      1
                                        NVIDIA Quadro K2000                       32  Windows Pro 6.1        1920x1080                      1
Core(TM) i7-4790K                       Intel(R) HD Graphics 4600                 8   Windows Pro 6.1        1440x900                       1
                                        Mirror                                    32  Windows Pro 6.1        1440x900                       1
                                                                                                             1920x1080                      1
                                        NVIDIA GeForce GTX 750 Ti                 16  Windows Pro 6.1        1920x1080                      1
Core(TM) i7-4770                        NVIDIA Quadro K600                        8   Windows Pro 6.1        1600x1200                      1
Core(TM) i7-4790K                       NVIDIA GeForce GTX 750 Ti                 32  Windows Pro 6.1        1920x1080                      1
Core(TM) i7-477

In [368]:
#fake[fake['brand']=='Core(TM) i7-4790'].sort_values('GC')
#pd.DataFrame(fake.groupby(['brand','GC','RAM','OS2','Screen']).labelK15.value_counts()).sort_values('labelK15')
pd.DataFrame(fake.groupby(['brand','GC','RAM','OS2','Screen']).labelP10.value_counts()).sort_values('labelP10')

labelP10
brand                                                                                GC                                        RAM OS2                        Screen                 labelP10          
Core(TM) i5-7500                                                                     Intel(R) HD Graphics 630                  8   Windows Ultimate 6.1       1366x768               4                1
Core(TM) i7-4770K                                                                    Microsoft Basic Display Adapter           8   Windows Pro 10.0           1280x1024              4                1
                                                                                     Intel(R) HD Graphics 4600                 8   Windows Pro 10.0           1920x1080              4                1
                                                                                                                               32  Windows Ultimate 6.1       1920x1080              4                1
                                                                                     AMD(ATI) Radeon HD 5450                   16  Windows Pro 6.1            (1920x1080, 1920x1080) 4                1
Core(TM) i7-4770HQ                                                                   Intel(R) Iris(TM) Pro                     16  Mac 10.12                  1440x900               2                1
                                                                                                                                   Mac 10.11                  1920x1080              2                1
                                                                                                                                                              1440x900               2                1
Core(TM) i7-4770K                                                                    Microsoft Basic Display Adapter           8   Windows Pro 10.0           1600x1200              4                1
Core(TM) i7-4770                                                                     VMware SVGA 3D                            3   Windows Entreprise 10.0    1440x900               4                1
                                                                                     NVIDIA Quadro K620                        16  Windows Pro 6.1            (1920x1080, 1920x1080) 4                1
                                                                                     NVIDIA Quadro K600                        8   Windows Pro 6.1            1920x1200              4                1
                                                                                                                                                              1600x1200              4                1
                                                                                                                               32  Windows Pro 6.1            1920x1080              4                1
                                                                                                                               16  Windows Pro 6.1            1920x1080              4                1
                                                                                                                                                              1600x900               4                1
                                                                                                                                   Windows Entreprise 10.0    1920x1080              4                1
                                                                                     Remote                                    8   Windows Pro 6.1            1920x1200              4                1
                                                                                     NVIDIA Quadro K600                        12  Windows Pro 6.1            1920x1080              4         

In [366]:
pd.DataFrame(fake.groupby(['brand','GC','RAM','OS2','Screen']).labelK15.count()).sort_values('labelK15')

labelK15
brand                                                                                GC                                        RAM OS2                        Screen                          
Core(TM) i5-7500                                                                     Intel(R) HD Graphics 630                  8   Windows Ultimate 6.1       1366x768                       1
Core(TM) i7-4770K                                                                    Microsoft Basic Display Adapter           8   Windows Pro 10.0           1280x1024                      1
                                                                                     Intel(R) HD Graphics 4600                 8   Windows Pro 10.0           1920x1080                      1
                                                                                                                               32  Windows Ultimate 6.1       1920x1080                      1
                                                                                     AMD(ATI) Radeon HD 5450                   16  Windows Pro 6.1            (1920x1080, 1920x1080)         1
Core(TM) i7-4770HQ                                                                   Intel(R) Iris(TM) Pro                     16  Mac 10.12                  1440x900                       1
                                                                                                                                   Mac 10.11                  1920x1080                      1
                                                                                                                                                              1440x900                       1
Core(TM) i7-4770K                                                                    Microsoft Basic Display Adapter           8   Windows Pro 10.0           1600x1200                      1
Core(TM) i7-4770                                                                     VMware SVGA 3D                            3   Windows Entreprise 10.0    1440x900                       1
                                                                                     NVIDIA Quadro K620                        16  Windows Pro 6.1            (1920x1080, 1920x1080)         1
                                                                                     NVIDIA Quadro K600                        8   Windows Pro 6.1            1920x1200                      1
                                                                                                                                                              1600x1200                      1
                                                                                                                               32  Windows Pro 6.1            1920x1080                      1
                                                                                                                               16  Windows Pro 6.1            1920x1080                      1
                                                                                                                                                              1600x900                       1
                                                                                                                                   Windows Entreprise 10.0    1920x1080                      1
                                                                                     Remote                                    8   Windows Pro 6.1            1920x1200                      1
                                                                                     NVIDIA Quadro K600                        12  Windows Pro 6.1            1920x1080                      1
Core(TM) i7-4770K                                                                    NVIDIA GeForce GT 730                     8   Windows Ultimate 6.1       1440x900     

In [370]:
fake.labelP1.value_counts()

2     7458
1      888
11      46
10      30
5       25
3       23
13      22
6       14
12      14
4       10
0        9
7        8
9        4
14       3
8        2
Name: labelP1, dtype: int64

In [194]:
#user.sort_values('GC')

In [15]:
from sklearn.cluster import AgglomerativeClustering
hc=AgglomerativeClustering(n_clusters=30, linkage='average',affinity='manhattan')
hc.fit(user)

AgglomerativeClustering(affinity='manhattan', compute_full_tree='auto',
            connectivity=None, linkage='average', memory=None,
            n_clusters=30,
            pooling_func=<function mean at 0x0000022DFA3DE840>)

In [339]:
fake['labelP30']=hc.labels_

In [340]:
fake[fake['brand']=='Core(TM) i7-4790'].sort_values('GC')

,brand,GC,RAM,OS2,Screen,labelP,labelP1,labelP2,labelP10,CGlabel,labelP30
352704,Core(TM) i7-4790,AMD(ATI) FirePro V3900 (FireGL V) Adapter,16,Windows Pro 6.1,"(1680x1050, 1680x1050)",0,2,7,4,1,0
267437,Core(TM) i7-4790,AMD(ATI) FirePro W2100 (FireGL V) Adapter,16,Windows Pro 6.1,"(2560x1440, 2560x1440)",0,2,7,4,1,0
599472,Core(TM) i7-4790,AMD(ATI) FirePro W5100,16,Windows Pro 6.3,1920x1080,0,2,7,4,1,0
158776,Core(TM) i7-4790,AMD(ATI) Radeon HD 4300/4500,6,Windows Pro 10.0,1920x1080,0,2,7,4,1,0
515634,Core(TM) i7-4790,AMD(ATI) Radeon HD 5450,3,Windows Ultimate 6.1,1366x768,5,1,12,4,1,0
75241,Core(TM) i7-4790,AMD(ATI) Radeon HD 5450,32,Windows Ultimate 6.1,1440x900,5,1,12,4,1,0
62068,Core(TM) i7-4790,AMD(ATI) Radeon HD 5800,16,Windows Pro 10.0,1920x1080,0,2,7,4,1,0
760411,Core(TM) i7-4790,AMD(ATI) Radeon HD 7000,24,Windows Pro 6.1,1366x768,0,2,7,4,1,0
706706,Core(TM) i7-4790,AMD(ATI) Radeon HD 7500,8,Windows Pro 10.0,1600x900,0,2,7,4,1,0
429763,Core(TM) i7-4790,AMD(ATI) Radeon HD 8350,4,Windows Ultimate 6.1,1920x1080,0,2,7,4,1,0


In [232]:
fake.groupby('labelP').apply(lambda x: x.apply(lambda s: s.value_counts().index[0],axis=0))


,brand,GC,RAM,OS2,Screen,labelP,labelP1
labelP,,,,,,,
0,Core(TM) i7-6700,Intel(R) HD Graphics 530,8,Windows Pro 10.0,1920x1080,0,2
1,Core(TM) i7- Q 740,AMD(ATI) Radeon HD 7400M,3,Windows Ultimate 6.1,1600x900,1,0
2,Xeon(R) E5-1603 0,NVIDIA Quadro K2100M,32,Windows Pro 10.0,1680x1050,2,6
3,Core(TM) M-5Y10c,Intel(R) HD Graphics 5300,4,Windows Home 10.0,1680x1050,3,4
4,Core(TM) i7-4870HQ,NVIDIA GeForce GTX 950,16,Windows Pro 10.0,1440x900,4,10
5,Core(TM) i5-3470,Intel(R) HD Graphics,4,Windows Ultimate 6.1,1920x1080,5,1
6,Pentium(R) Dual-Core E5200,Intel(R) G41 Express Chipset,3,Windows Ultimate 6.1,1600x900,6,3
7,Core(TM) i7- 930,AMD(ATI) Radeon HD 5800,3,Windows Entreprise 10.0,2560x1440,7,14
8,Xeon(R) W3520,NVIDIA Quadro 4000,8,Windows Pro 10.0,1920x1080,8,12


In [342]:
fake.labelP30.value_counts()

0     5308
28    1682
3     1025
19     117
6       94
5       48
4       44
1       36
22      20
13      19
12      19
9       14
14      13
8       13
17      13
27      11
2       10
18       8
16       8
23       6
11       6
20       6
29       6
10       5
25       5
24       5
7        5
15       4
26       3
21       3
Name: labelP30, dtype: int64

In [346]:
fake[fake.labelP30==28].sort_values('GC')

,brand,GC,RAM,OS2,Screen,labelP,labelP1,labelP2,labelP10,CGlabel,labelP30
290917,Core(TM) i5-3340M,AMD (ATI) FirePro M6000 (FireGL V) Mobility Pro,8,Windows Pro 10.0,1600x900,0,2,7,4,1,28
74765,Core(TM) i5-2410M,AMD(ATI) Radeon HD 6470M,8,Windows Ultimate 6.1,1366x768,0,2,7,4,1,28
213377,Core(TM) i5-2410M,AMD(ATI) Radeon HD 6750M,4,Windows Ultimate 6.1,1366x768,0,2,7,4,1,28
419029,Core(TM) i5-2450M,AMD(ATI) Radeon HD 7400M,3,Windows Ultimate 6.1,1366x768,1,0,2,4,1,28
53948,Core(TM) i5-2410M,AMD(ATI) Radeon HD 7400M,4,Windows Pro 6.1,1600x900,0,2,7,4,1,28
419025,Core(TM) i5-2450M,AMD(ATI) Radeon HD 7400M,3,Windows Ultimate 6.1,1366x768,1,0,2,4,1,28
397402,Core(TM) i7-3632QM,AMD(ATI) Radeon HD 7500M/7600M,8,Windows Ultimate 6.1,1366x768,0,2,7,4,1,28
389588,Core(TM) i5-3340M,AMD(ATI) Radeon HD 7500M/7600M,8,Windows Pro 6.1,1600x900,0,2,7,4,1,28
517626,Core(TM) i7-3632QM,AMD(ATI) Radeon HD 7500M/7600M,8,Windows Home 10.0,1920x1080,0,2,7,4,1,28
680098,Core(TM) i5-3230M,AMD(ATI) Radeon HD 7600M,6,Windows Pro 10.0,1366x768,0,2,7,4,1,28


In [255]:
fake[fake.labelP10==1].sort_values('GC')
#fake[fake.GC=='AMD(ATI) Radeon HD 4600']

,brand,GC,RAM,OS2,Screen,labelP,labelP1,labelP2,labelP10
560197,Xeon(R) E5-1603 0,AMD(ATI) FirePro V4900 (FireGL V),8,Windows Entreprise 6.1,1280x1024,2,6,5,1
436570,Xeon(R) E5-1603 0,AMD(ATI) FirePro V5900 (FireGL V) Adapter,16,Windows Pro 10.0,1280x1024,2,6,5,1
320731,Xeon(R) E5-1603 0,AMD(ATI) FirePro V7900,32,Windows Pro 6.1,"(1920x1200, 1920x1200)",2,6,5,1
158556,Xeon(R) X5550,AMD(ATI) FirePro V7900,32,Windows Ultimate 6.1,1680x1050,2,6,5,1
756326,AMD Athlon(tm) II X4 640 Processor,AMD(ATI) Radeon HD 5400,4,Windows Home 6.1,1440x900,3,4,0,1
209159,Core(TM)2 Quad Q9550,AMD(ATI) Radeon HD 5400,8,Windows Pro 10.0,1680x1050,3,4,0,1
125440,Pentium(R) G3260,AMD(ATI) Radeon R5 200,3,Windows Ultimate 6.1,1440x900,3,4,0,1
576276,AMD FX(tm)-6350 Six-Core Processor,AMD(ATI) Radeon R5 200,4,Windows Home 10.0,1680x1050,3,4,0,1
433417,Pentium(R) Dual-Core E5700,Intel(R) G45/G43 Express Chipset,4,Windows Home 6.1,1680x1050,3,4,0,1
434557,Core(TM) M-5Y10c,Intel(R) HD Graphics 5300,4,Windows Home 10.0,1920x1080 Touch,3,4,0,1


In [214]:
fake.GC.value_counts()

Intel(R) HD Graphics 530                           725
Intel(R) HD Graphics 4600                          595
Intel(R) HD Graphics 520                           541
Intel(R) HD Graphics 630                           498
Intel(R) HD Graphics 620                           487
Intel(R) HD Graphics                               396
Intel(R) HD Graphics Family                        313
Intel(R) HD Graphics 4000                          300
Intel(R) HD Graphics 5500                          289
NVIDIA GeForce GT 730                              255
NVIDIA Quadro K620                                 210
NVIDIA GeForce GTX 750 Ti                          178
Intel(R) HD Graphics 4400                          119
NVIDIA GeForce GTX 1050 Ti                         119
Mirror                                             117
Microsoft Basic Display Adapter                    115
NVIDIA GeForce GTX 1050                            114
Intel(R) HD Graphics 3000                          113
NVIDIA Qua

In [236]:
fake[fake.GC=='Intel(R) HD Graphics 630'].labelP.value_counts()

0    498
Name: labelP, dtype: int64

In [256]:
fake.RAM.value_counts()

8     3674
16    1894
4     1550
32     427
3      316
6      231
12     228
7      102
2       85
24      49
Name: RAM, dtype: int64

In [322]:
##explore
#CG=fake[['brand','GC']]
CG=pd.get_dummies(fake,columns = ['brand','GC']).drop(['RAM','OS2','Screen','labelP','labelP1','labelP2','labelP10','CGlabel'],axis=1)
CG.shape

(8556, 719)

In [323]:
CG.head()

,"brand_('Intel(R) Xeon(R) CPU E5-2667 0 @ 2.90GHz', 'Intel(R) Xeon(R) CPU E5-2667 0 @ 2.90GHz')","brand_('Intel(R) Xeon(R) CPU E5620 @ 2.40GHz', 'Intel(R) Xeon(R) CPU E5620 @ 2.40GHz')",brand_AMD A10-5745M Radeon HD,brand_AMD A10-5750M Radeon HD,brand_AMD A10-6800K Radeon HD,brand_AMD A10-7300 Radeon R6,brand_AMD A10-7800 Radeon R7,brand_AMD A10-7850K Radeon R7,brand_AMD A10-7860K Radeon R7,brand_AMD A10-8700P Radeon R6,...,GC_NVIDIA Quadro P1000,GC_NVIDIA Quadro P2000,GC_NVIDIA Quadro P400,GC_NVIDIA Quadro P4000,GC_NVIDIA Quadro P5000,GC_NVIDIA Quadro P600,GC_Remote,GC_Standard VGA Graphics Adapter,GC_VMware SVGA 3D,GC_Wireless
292662,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
336254,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
583621,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18118,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
476980,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [324]:
#hc=AgglomerativeClustering(n_clusters=10, linkage='average',affinity='manhattan')
hc1=AgglomerativeClustering(n_clusters=10, linkage='average',affinity='manhattan')

hc1.fit(CG)

AgglomerativeClustering(affinity='manhattan', compute_full_tree='auto',
            connectivity=None, linkage='average', memory=None,
            n_clusters=10,
            pooling_func=<function mean at 0x0000022DFA3DE840>)

In [325]:
fake['CGlabel']=hc1.labels_

In [326]:
fake[fake['brand']=='Core(TM) i7-4790'].sort_values('GC')

,brand,GC,RAM,OS2,Screen,labelP,labelP1,labelP2,labelP10,CGlabel
352704,Core(TM) i7-4790,AMD(ATI) FirePro V3900 (FireGL V) Adapter,16,Windows Pro 6.1,"(1680x1050, 1680x1050)",0,2,7,2,1
267437,Core(TM) i7-4790,AMD(ATI) FirePro W2100 (FireGL V) Adapter,16,Windows Pro 6.1,"(2560x1440, 2560x1440)",0,2,7,2,1
599472,Core(TM) i7-4790,AMD(ATI) FirePro W5100,16,Windows Pro 6.3,1920x1080,0,2,7,2,1
158776,Core(TM) i7-4790,AMD(ATI) Radeon HD 4300/4500,6,Windows Pro 10.0,1920x1080,0,2,7,2,1
515634,Core(TM) i7-4790,AMD(ATI) Radeon HD 5450,3,Windows Ultimate 6.1,1366x768,5,1,12,4,1
75241,Core(TM) i7-4790,AMD(ATI) Radeon HD 5450,32,Windows Ultimate 6.1,1440x900,5,1,12,4,1
62068,Core(TM) i7-4790,AMD(ATI) Radeon HD 5800,16,Windows Pro 10.0,1920x1080,0,2,7,2,1
760411,Core(TM) i7-4790,AMD(ATI) Radeon HD 7000,24,Windows Pro 6.1,1366x768,0,2,7,2,1
706706,Core(TM) i7-4790,AMD(ATI) Radeon HD 7500,8,Windows Pro 10.0,1600x900,0,2,7,2,1
429763,Core(TM) i7-4790,AMD(ATI) Radeon HD 8350,4,Windows Ultimate 6.1,1920x1080,0,2,7,2,1


In [327]:
fake.CGlabel.value_counts()

1    8451
0      43
3      24
2      19
7       4
4       4
8       4
6       3
5       3
9       1
Name: CGlabel, dtype: int64

In [274]:
fake.groupby(['brand','GC']).count().sort_values('RAM')

RAM  \
brand                                                                                GC                                                     
Core(TM) i7- 870                                                                     NVIDIA Quadro K620                                 1   
Core(TM) i7-3770K                                                                    NVIDIA GeForce GTX 550 Ti                          1   
                                                                                     NVIDIA GeForce GT 640                              1   
                                                                                     NVIDIA GeForce GT 630                              1   
                                                                                     NVIDIA GeForce GT 420                              1   
                                                                                     AMD(ATI) FirePro V4900 (FireGL V)                  1   
Core(TM) i7-3770                                                                     Standard VGA Graphics Adapter                      1   
                                                                                     NVIDIA Quadro P600                                 1   
                                                                                     NVIDIA NVS 310                                     1   
                                                                                     NVIDIA GeForce GTX 960                             1   
Core(TM) i7-3770K                                                                    NVIDIA GeForce GTX 560                             1   
Core(TM) i7-3770                                                                     NVIDIA GeForce GTX 750                             1   
                                                                                     NVIDIA GeForce GTX 660 Ti                          1   
                                                                                     NVIDIA GeForce GTX 660                             1   
                                                                                     NVIDIA GeForce GTX 650 Ti BOOST                    1   
                                                                                     NVIDIA GeForce GTX 1060 3GB                        1   
                                                                                     NVIDIA GeForce GTX 1050 Ti                         1   
                                                                                     NVIDIA GeForce GTS 250                             1   
                                                                                     NVIDIA GeForce GT 740                              1   
                                                                                     NVIDIA GeForce GT 730                              1   
                                                                                     NVIDIA GeForce GT 635                              1   
                                                                                     NVIDIA GeForce GTX 670                             1   
                                                                                     NVIDIA GeForce GT 440                              1   
Core(TM) i7-3770K                                                                    NVIDIA GeForce GTX 650                             1   
                                                                                     NVIDIA GeForce GTX 750 Ti                          1   
Core(TM) i7-4700MQ                                                                   NVIDIA GeForce GT 755M                             1   
                                                                                     NVIDIA GeForce GT 750M                             1   
Core(TM) i7-4700HQ                           

In [328]:
fake.groupby('CGlabel').apply(lambda x: x.apply(lambda s: s.value_counts().index[0],axis=0))


,brand,GC,RAM,OS2,Screen,labelP,labelP1,labelP2,labelP10,CGlabel
CGlabel,,,,,,,,,,
0,AMD A6-7310 Radeon R4,AMD(ATI) Radeon R4,7,Windows Home 10.0,1366x768,0,2,7,2,0
1,Core(TM) i7-6700,Intel(R) HD Graphics 530,8,Windows Pro 10.0,1920x1080,0,2,7,2,1
2,Core(TM) i5-5250U,Intel(R) HD Graphics 6000,8,Windows Entreprise 10.0,1440x900,11,5,1,3,2
3,Core(TM) i7-8550U,Intel(R) UHD Graphics 620,8,Windows Home 10.0,1920x1080,0,2,7,2,3
4,AMD A6-5200 Radeon HD,AMD(ATI) Radeon HD 8400 / R3,3,Windows Home 10.0,1366x768,0,2,7,2,4
5,AMD E2-1800 Radeon HD,AMD(ATI) Radeon HD 7340,4,Windows Pro 6.2,1366x768,0,2,7,2,5
6,AMD A8-5550M Radeon HD,AMD(ATI) Radeon HD 8550G,7,Windows Pro 10.0,1600x900,0,2,7,2,6
7,Core(TM) M-5Y10c,Intel(R) HD Graphics 5300,4,Windows Home 10.0,1920x1080 Touch,3,4,0,1,7
8,AMD E1-1200 Radeon HD,AMD(ATI) Radeon HD 7310,3,Windows Pro 6.1,1366x768,0,2,7,2,8
